<a href="https://colab.research.google.com/github/gibsonx/jlpt_simulator/blob/dev/graphs/n3/outliner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/gibsonx/jlpt_simulator.git
    %cd jlpt_simulator
    !git checkout dev
    
    !apt-get install python3-dev graphviz libgraphviz-dev pkg-config
    !pip install -r requirements.txt
else:
  print('Not running on CoLab')


Not running on CoLab


In [17]:

import json
import logging
import random
import time

import pandas as pd
import yaml
from tqdm import tqdm

# from langchain_community.embeddings import XinferenceEmbeddings
from libs.LLMs import *

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

from graphs.n3.ExamTaskHandler import ExamTaskHandler
load_dotenv()

True

In [18]:
def collect_vocabulary(file_path):
    # Read the CSV file
    data = pd.read_csv(file_path)
    # Shuffle the rows and reset the index
    words = data.iloc[:, :2].sample(frac=1).reset_index(drop=True)
    # Extract the second column (values) and convert to a single-line string
    vocab_string = ','.join(words.iloc[:, 1].astype(str).tolist())
    return vocab_string

In [19]:
# Import N3 Vocabulary
file_path = 'Vocab/n3.csv'
# Display the content of the CSV file
vocab_dict = collect_vocabulary(file_path)
with open("Vocab/topics.txt", "r", encoding="utf-8") as file:
    topics_list = [line.strip() for line in file]

In [20]:
vocab_dict

'まんぞく,かんかく,けっしん,スキー,ぎん,さからう,けいこく,プラス,さんそ,しゅちょう,とう,きんにく,くりかえす,する,むく,ごみ,めんどう,わがまま,しかく,ほり,しつど,かてい,がっき,じき,な,せんしゅ,ちがいない,とう,え,はってん,もとづく,てんねん,れつ,ところが,まぜる,めいわく,たすける,せい,しゅうだん,ようい,ながれ,うらぎる,みごと,どりょく,へんか,あずける,しっけ,できごと,トップ,くむ,めんきょ,ためす,しぼう,けっこう,ひょうばん,はなす,げんしょう,さめる,はっぴょう,だまる,せつ,かた,はずす,ほう,いまにも,ひざ,なまける,わん,きのどく,かくじつ,せいき,たんに,かせぐ,やく,よごす,はいゆう,くつう,かく,もと,せめる,たまたま,いったい,しりあい,むだ,ごがく,もと,せつやく,いけない,やとう,さく,たいせん,してん,へる,げんじつ,うちゅう,おひる,ちえ,ビール,かたい,したがう,じんるい,おそらく,わずか,はね,ふで,チーズ,りょう,しはらう,じかに,きょうふ,むかえ,テント,たから,ますます,ひん,すませる,つい,かがやく,しゅうにゅう,うけとる,しき,りょうがえ,どうぞよろしく,たちば,まちがい,あんまり,しはい,きゅうそく,ひづけ,きんし,える,よみ,どういつ,あたたまる,さそう,さい,ことわざ,はんざい,はくぶつかん,あたたかい,じじつ,カー,ゆう,よせる,ヨット,ちゃ,ふ,えん,ちる,せい,さかり,まさか,こい,しんこく,せいき,し,きゅうけい,さる,きらう,ざいさん,きかい,かわる,まぶしい,けん,もしかすると,けんとう,のせる,つめる,こうほ,で,おおう,こうふく,クリーム,ひっこし,へらす,しょうにん,みとめる,きれ,さら,あいする,せいちょう,つうがく,かえる,きみ,しき,すむ,あける,ダイヤ,おせん,てき,たいし,にっこり,むちゅう,ウイスキー,かしこい,せいかく,はい,しぼう,しゅつじょう,そのうち,ぜん,きんちょう,よろしく (かん),よそく,せいふ,せいり,ヨーロッパ,うず,ばくだい,しんごう,へる,ふたたび,さんか,つく,ドラマ,とかい,しんせん,こくご,よのなか,きちょう,ちらす,けいこう,かける,あたり,くだる,きこう,せいぞう,いためる,かえる,ねったい,ちゃんと,おさめる,ただ

#### load Models

In [21]:
from typing import List, Optional

from langchain_core.prompts import ChatPromptTemplate

from pydantic import BaseModel, Field

# instruction = """
# Section 1: Vocabulary and Grammar
# - Kanji reading (kanji_reading): 8 questions
# - Write Chinese characters (write_chinese): 6 questions
# - Word Meaning Selection (word_meaning): 11 questions
# - Synonyms substitution (synonym_substitution): 5 questions
# - word usage (word_usage): 5 questions
# - Grammar fill in the blank (sentence_grammar): 13 questions
# - Sentence sorting (sentence_sort): 5 questions
# - Grammar structure selection (sentence_structure): 4-5 questions
#
# Section 2: Reading Comprehension
# - Short passages (short_passage_read): 4 articles
# - Mid-size passages (midsize_passage_read): 2 articles
# - Long passages (long_passage_read): 1 articles
# - Information retrieval (info_retrieval): 1 articles
#
# Section 3: Listening Comprehension
# - Topic understanding (topic_understanding): 6 questions
# - Key understanding (keypoint_understanding): 6 questions
# - Summary understanding (summary_understanding): 3 questions
# - Active expression (active_expression): 4 questions
# - Immediate acknowledgment (immediate_ack): 9 questions
# """

instruction = """
Section 1: Vocabulary and Grammar
- Kanji reading (kanji_reading): 8 questions

Section 2: Reading Comprehension
- Information retrieval (info_retrieval): 0 articles

Section 3: Listening Comprehension
- Topic understanding (topic_understanding): 0 questions

"""


direct_gen_outline_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"You are a japanese teacher. Your job is to write an outline for a JLPT Japanese-Language Proficiency Test N3 level exam paper. The words in outline must be presented in Japanese only. The complexity should be restricted to N3 level and respect japanese culture. The JLPT Japanese-Language Proficiency Test exam paper includes a mix of easy, moderate, and difficult questions to accurately assess the test-taker's proficiency across different aspects of the language."
            f"First,  randomly pick Japanese words in 'Vocabulary' for questions in Vocabulary and Grammar. At Section 2 and 3, randomly choose topics in 'TopicList' for Reading Comprehension and Listening Comprehension Sections, don't repeat to choose a same word or topic"
            f"Second, you should abide by the provided exam instruction and decide the number of questions and content in the each Section."
            f"Finally, write the outline of the examination paper in japanese and provide question topics according to the instructions."
            f"Instruction: {instruction}",
        ),
        ("user", "TopicList: {topic_list}, Vocabulary: {vocab_dict}"),
    ]
)


## Data Strcuture

In [22]:
class QuestionTopic(BaseModel):
    topic: str = Field(..., title="a vocabulary or topic hint for a question")


class Subsection(BaseModel):
    subsection_title: str = Field(..., title="subsection English word in () only from the instruction")
    description: str = Field(..., title="giving the number of questions and requirements")
    question_topics: Optional[List[QuestionTopic]] = Field(
        default_factory=list
    )

    @property
    def as_str(self) -> str:
        question_topics_str = "\n".join(
            f"- **{qt.topic}**" for qt in self.question_topics
        )
        return f"### {self.subsection_title}\n\n{self.description}\n\n{question_topics_str}".strip()

class Section(BaseModel):
    section_title: str = Field(..., title="Title of the section")
    subsections: Optional[List[Subsection]] = Field(
        default_factory=list,
        title="Titles and reason for each subsection of the JLPT exam page.",
    )

    @property
    def as_str(self) -> str:
        subsections = "\n\n".join(
            subsection.as_str for subsection in self.subsections or []
        )
        return f"## {self.section_title}\n\n{subsections}".strip()


class Outline(BaseModel):
    page_title: str = Field(..., title="Title of the JLPT exam page")
    sections: List[Section] = Field(
        default_factory=list,
        title="Titles and descriptions for each section of the JLPT exam paper.",
    )

    @property
    def as_str(self) -> str:
        sections = "\n\n".join(section.as_str for section in self.sections)
        return f"# {self.page_title}\n\n{sections}".strip()


In [23]:
# Read the topics from a file, sort them, and print the sorted list
def process_topics(file_path):
    try:
        # Read the file
        with open(file_path, 'r', encoding='utf-8') as file:
            topics = file.readlines()

        # Remove any extra whitespace or newline characters
        topics = [topic.strip() for topic in topics if topic.strip()]

        # Shuffle the topics randomly
        random.shuffle(topics)

    except FileNotFoundError:
        print("The file was not found. Please check the file path.")
    except Exception as e:
        print("An error occurred:", str(e))

    except FileNotFoundError:
        print("The file was not found. Please check the file path.")
    except Exception as e:
        print("An error occurred:", str(e))

In [24]:
# Preload all topics from the file
with open("Vocab/topics.txt", "r", encoding="utf-8") as file:
    topics_list = [line.strip() for line in file]

generate_outline_direct = direct_gen_outline_prompt | azure_llm.with_structured_output(Outline)
initial_outline = generate_outline_direct.invoke({"topic_list": topics_list, "vocab_dict": vocab_dict})

2025-05-27 10:24:27,526 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


In [25]:
from IPython.display import display, Markdown

display(Markdown(initial_outline.as_str))

# 日本語能力試験N3 模擬試験 問題用紙

## 第1部　語彙・文法

### 漢字読み (kanji_reading)

8問。与えられた漢字の読みをひらがなで答えなさい。

- **満足**
- **繰り返す**
- **筋肉**
- **努力**
- **選手**
- **現象**
- **助ける**
- **情報**

In [26]:
outliner_json = initial_outline.model_dump_json()
data = json.loads(outliner_json)  # Replace with your actual JSON data

start_time = time.time()

output_data = {'sections': []}

for section in data['sections']:
    output_section = {'section_title': section['section_title'], 'subsections': []}

    for subsection in tqdm(section['subsections'], desc=f"Processing {section['section_title']}"):
        function_name = subsection['subsection_title'].split(' (')[1].rstrip(')')
        questions = subsection['question_topics']

        for question in tqdm(questions, desc=f"Processing {subsection['subsection_title']}"):
            handler = ExamTaskHandler()
            func = getattr(handler, function_name, None)

            if func:
                max_attempts = 2
                original_topic = question['topic']  # Optional: Track original topic
                for attempt in range(max_attempts):
                    try:
                        result = func(question['topic'])
                        question['result'] = result
                        break  # Exit on success
                    except Exception as e:
                        if attempt < max_attempts - 1:
                            # Replace topic and retry
                            question['topic'] = random.choice(topics_list)
                        else:
                            question['result'] = f"Error after {max_attempts} attempts: {str(e)}"
            else:
                question['result'] = f"Method {function_name} not found"

        output_subsection = {
            'subsection_title': subsection['subsection_title'],
            'description': subsection['description'],
            'question_topics': questions
        }
        output_section['subsections'].append(output_subsection)

    output_data['sections'].append(output_section)

# End the timer
end_time = time.time()

# Calculate the total execution time
execution_time = end_time - start_time

print(f"Total execution time: {execution_time:.2f} seconds")

Processing 漢字読み (kanji_reading):   0%|          | 0/8 [00:00<?, ?it/s]

---WEB SEARCH---
---Generator----


2025-05-27 10:24:33,787 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-05-27 10:24:33,789 - INFO - Generated message: <ol>
    <li>
        旅行がとても楽しくて<u>満足</u>しています。
        <ol type="1">
            <li>まんそく</li>
            <li>まんぞく</li>
            <li>まんしょく</li>
            <li>まんしょく</li>
        </ol>
    </li>
</ol>


---REVISOR---


2025-05-27 10:24:38,401 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-05-27 10:24:38,403 - INFO - Refelect message: Thank you for your submission. Here is a detailed critique and recommendations:

1. Content Accuracy:
- The vocabulary question accurately targets the JLPT N3 level. "満足" is a common word at this level, and testing its reading is appropriate.
- The sentence context, "旅行がとても楽しくて満足しています," is clear, natural, and relevant to everyday situations, which suits the N3 level.

2. Question and Answer Quality:
- The question format (choosing the correct reading of the underlined word) is straightforward and suitable for assessing vocabulary reading skills.
- However, there is a noticeable issue with the answer choices: options 3 and 4 are identical ("まんしょく"). This redundancy can confuse test-takers and does not provide a meaningful distractor

...

---REVISOR---


2025-05-27 10:26:30,089 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-05-27 10:26:30,092 - INFO - Refelect message: Content Accuracy:
- The vocabulary word "情報" remains appropriate for JLPT N3 level.
- The context sentence "ニュースで新しい情報を知りました。" is clear, natural, and relevant, providing a realistic usage scenario.
- The answer choices include one correct reading "じょうほう" and three distractors that are plausible as misreadings, which is effective for testing reading skills.

Question and Answer Quality:
- The question is clearly presented and unambiguous.
- The distractors are reasonable and challenge the learner to distinguish the correct reading.
- The difficulty level aligns well with N3 expectations, focusing on kanji reading recognition in context.

Cultural and Academic Relevance:
- The sentence and vocabulary are culturally appropriate and co

--- AI Reviser feels Good Enough ---
--- Formatter ---


2025-05-27 10:26:31,068 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-05-27 10:26:31,071 - INFO - Formatted message: {'html_question': 'ニュースで新しい<u>情報</u>を知りました。', 'correct_answer': 1, 'choices': ['じょうほう', 'じょうほ', 'じょうこう', 'じょうほく']}

Processing 第1部　語彙・文法: 100%|██████████| 1/1 [02:03<00:00, 123.52s/it]

Total execution time: 123.52 seconds


In [27]:
yaml_str = yaml.dump(output_data, allow_unicode=True, sort_keys=False, default_flow_style=False)
# Display as formatted YAML
display(Markdown(yaml_str))

sections:
- section_title: 第1部　語彙・文法
  subsections:
  - subsection_title: 漢字読み (kanji_reading)
    description: 8問。与えられた漢字の読みをひらがなで答えなさい。
    question_topics:
    - topic: 満足
      result:
        html_question: 旅行がとても楽しくて<u>満足</u>しています。
        correct_answer: 2
        choices:
        - まんそく
        - まんぞく
        - まんしょく
        - みつぞく
    - topic: 繰り返す
      result:
        html_question: 先生は大切なことを<u>繰り返して</u>説明した。
        correct_answer: 1
        choices:
        - くりかえして
        - くりかえした
        - くりかいして
        - くるかえして
    - topic: 筋肉
      result:
        html_question: 毎日運動すると<u>筋肉</u>が強くなります。
        choices:
        - きんいく
        - きんにく
        - すじにく
        - けんにく
        correct_answer: 2
    - topic: 努力
      result:
        html_question: 目標を達成するために毎日<u>努力</u>しています。
        correct_answer: 1
        choices:
        - どりょく
        - どりく
        - ぬりょく
        - ぬりく
    - topic: 選手
      result:
        html_question: 彼はサッカーの<u>選手</u>です。
        correct_answer: 2
        choices:
        - せんしゃ
        - せんしゅ
        - せんじゅ
        - せんしょ
    - topic: 現象
      result:
        html_question: 最近、この地域で珍しい<u>現象</u>が観察されています。
        correct_answer: 1
        choices:
        - げんしょう
        - げんぞう
        - けんしょう
        - けんぞう
    - topic: 助ける
      result:
        html_question: 困っている人を<u>助ける</u>のは大切です。
        correct_answer: 2
        choices:
        - てつだう
        - たすける
        - ささえる
        - まもる
    - topic: 情報
      result:
        html_question: ニュースで新しい<u>情報</u>を知りました。
        correct_answer: 1
        choices:
        - じょうほう
        - じょうほ
        - じょうこう
        - じょうほく


In [28]:
output_data

{'sections': [{'section_title': '第1部\u3000語彙・文法',
   'subsections': [{'subsection_title': '漢字読み (kanji_reading)',
     'description': '8問。与えられた漢字の読みをひらがなで答えなさい。',
     'question_topics': [{'topic': '満足',
       'result': {'html_question': '旅行がとても楽しくて<u>満足</u>しています。',
        'correct_answer': 2,
        'choices': ['まんそく', 'まんぞく', 'まんしょく', 'みつぞく']}},
      {'topic': '繰り返す',
       'result': {'html_question': '先生は大切なことを<u>繰り返して</u>説明した。',
        'correct_answer': 1,
        'choices': ['くりかえして', 'くりかえした', 'くりかいして', 'くるかえして']}},
      {'topic': '筋肉',
       'result': {'html_question': '毎日運動すると<u>筋肉</u>が強くなります。',
        'choices': ['きんいく', 'きんにく', 'すじにく', 'けんにく'],
        'correct_answer': 2}},
      {'topic': '努力',
       'result': {'html_question': '目標を達成するために毎日<u>努力</u>しています。',
        'correct_answer': 1,
        'choices': ['どりょく', 'どりく', 'ぬりょく', 'ぬりく']}},
      {'topic': '選手',
       'result': {'html_question': '彼はサッカーの<u>選手</u>です。',
        'correct_answer': 2,
        'choices':

In [29]:
def render_to_html(sections):
    html = '<html><body>\n'
    for section in sections:
        html += f'<h1>{section.get("section_title", "")}</h1>\n'
        for subsection in section.get("subsections", []):
            html += f'<h2>{subsection.get("subsection_title", "")}</h2>\n'
            if "description" in subsection:
                html += f'<p>{subsection["description"]}</p>\n'
            for qtopic in subsection.get("question_topics", []):
                html += f'<h3>{qtopic.get("topic", "")}</h3>\n'
                result = qtopic.get("result", {})
                if isinstance(result, dict):
                    if "html_article" in result:
                        html += result["html_article"] + "\n"
                    if "html_question" in result:
                        html += f'<p><strong>Q: {result["html_question"]}</strong></p>\n'
                        if "choices" in result:
                            correct = result.get("correct_answer", -1)
                            html += '<ul>\n'
                            for idx, choice in enumerate(result["choices"], 1):
                                if idx == correct:
                                    html += f"<li><b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"
                                else:
                                    html += f"<li>{choice}</li>\n"
                            html += '</ul>\n'
                        if "explanation" in result:
                            html += f'<p><em>Explanation: {result["explanation"]}</em></p>\n'
                    if "questions" in result and isinstance(result["questions"], list):
                        for q in result["questions"]:
                            html += f'<p><strong>Q: {q["question"]}</strong></p>\n'
                            if "choices" in q:
                                correct = q.get("correct_answer", -1)
                                html += '<ul>\n'
                                for idx, choice in enumerate(q["choices"], 1):
                                    if idx == correct:
                                        html += f"<li><b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"
                                    else:
                                        html += f"<li>{choice}</li>\n"
                                html += '</ul>\n'
                            if "explanation" in q:
                                html += f'<p><em>Explanation: {q["explanation"]}</em></p>\n'
                    if "questions" in result and isinstance(result["questions"], dict):
                        q = result["questions"]
                        html += f'<p><strong>Q: {q["question"]}</strong></p>\n'
                        if "choices" in q:
                            correct = q.get("correct_answer", -1)
                            html += '<ul>\n'
                            for idx, choice in enumerate(q["choices"], 1):
                                if idx == correct:
                                    html += f"<li><b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"
                                else:
                                    html += f"<li>{choice}</li>\n"
                            html += '</ul>\n'
                        if "explanation" in q:
                            html += f'<p><em>Explanation: {q["explanation"]}</em></p>\n'
    html += '</body></html>'
    return html

In [30]:
html_output = render_to_html(output_data['sections'])
display(Markdown(html_output))

<html><body>
<h1>第1部　語彙・文法</h1>
<h2>漢字読み (kanji_reading)</h2>
<p>8問。与えられた漢字の読みをひらがなで答えなさい。</p>
<h3>満足</h3>
<p><strong>Q: 旅行がとても楽しくて<u>満足</u>しています。</strong></p>
<ul>
<li>まんそく</li>
<li><b>まんぞく</b> <span style='color:green;'>(correct)</span></li>
<li>まんしょく</li>
<li>みつぞく</li>
</ul>
<h3>繰り返す</h3>
<p><strong>Q: 先生は大切なことを<u>繰り返して</u>説明した。</strong></p>
<ul>
<li><b>くりかえして</b> <span style='color:green;'>(correct)</span></li>
<li>くりかえした</li>
<li>くりかいして</li>
<li>くるかえして</li>
</ul>
<h3>筋肉</h3>
<p><strong>Q: 毎日運動すると<u>筋肉</u>が強くなります。</strong></p>
<ul>
<li>きんいく</li>
<li><b>きんにく</b> <span style='color:green;'>(correct)</span></li>
<li>すじにく</li>
<li>けんにく</li>
</ul>
<h3>努力</h3>
<p><strong>Q: 目標を達成するために毎日<u>努力</u>しています。</strong></p>
<ul>
<li><b>どりょく</b> <span style='color:green;'>(correct)</span></li>
<li>どりく</li>
<li>ぬりょく</li>
<li>ぬりく</li>
</ul>
<h3>選手</h3>
<p><strong>Q: 彼はサッカーの<u>選手</u>です。</strong></p>
<ul>
<li>せんしゃ</li>
<li><b>せんしゅ</b> <span style='color:green;'>(correct)</span></li>
<li>せんじゅ</li>
<li>せんしょ</li>
</ul>
<h3>現象</h3>
<p><strong>Q: 最近、この地域で珍しい<u>現象</u>が観察されています。</strong></p>
<ul>
<li><b>げんしょう</b> <span style='color:green;'>(correct)</span></li>
<li>げんぞう</li>
<li>けんしょう</li>
<li>けんぞう</li>
</ul>
<h3>助ける</h3>
<p><strong>Q: 困っている人を<u>助ける</u>のは大切です。</strong></p>
<ul>
<li>てつだう</li>
<li><b>たすける</b> <span style='color:green;'>(correct)</span></li>
<li>ささえる</li>
<li>まもる</li>
</ul>
<h3>情報</h3>
<p><strong>Q: ニュースで新しい<u>情報</u>を知りました。</strong></p>
<ul>
<li><b>じょうほう</b> <span style='color:green;'>(correct)</span></li>
<li>じょうほ</li>
<li>じょうこう</li>
<li>じょうほく</li>
</ul>
</body></html>

In [31]:
# timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
# filename = f"./output/jlpt_simulator/JLPT_{timestamp}.html"

# with open(filename, "w", encoding="utf-8") as file:
#     file.write(html_output)